# Donut-tiny OCR 데이터 준비

images 폴더 업로드

In [1]:
from google.colab import files
uploaded = files.upload()


Saving 20251113_알촌_8300.jpg to 20251113_알촌_8300.jpg
Saving 20251117_죠스아이스크림_1400.jpg to 20251117_죠스아이스크림_1400.jpg
Saving 20251117_죠스아이스크림_1500.jpg to 20251117_죠스아이스크림_1500.jpg
Saving 20251118_매머드익스프레스_1100.jpg to 20251118_매머드익스프레스_1100.jpg
Saving 20251118_매머드익스프레스_3400.jpg to 20251118_매머드익스프레스_3400.jpg
Saving 20251118_맥도날드_4200.jpg to 20251118_맥도날드_4200.jpg
Saving 20251118_맥도날드_4600.jpg to 20251118_맥도날드_4600.jpg
Saving 20251118_몬스터플레이스_22000.jpg to 20251118_몬스터플레이스_22000.jpg
Saving 20251118_은하수식당_9000.jpg to 20251118_은하수식당_9000.jpg
Saving 20251119_emarteveryday_8680.jpg to 20251119_emarteveryday_8680.jpg
Saving 20251119_몬스터플레이스_11000.jpg to 20251119_몬스터플레이스_11000.jpg
Saving 20251120_CU_2400.jpg to 20251120_CU_2400.jpg
Saving 20251120_CU_2700.jpg to 20251120_CU_2700.jpg
Saving 20251123_맥도날드_9100.jpg to 20251123_맥도날드_9100.jpg
Saving 20251123_캔모아_29000.jpg to 20251123_캔모아_29000.jpg
Saving 20251125_명랑핫도그_7200.jpg to 20251125_명랑핫도그_7200.jpg
Saving 20251125_몬스터플레이스_10500.jpg to 20251125_몬스터플

업로드된 이미지들을 dataset 구조로 정리

In [2]:
import os

# dataset 폴더 생성
os.makedirs("donut_dataset/images", exist_ok=True)

# 업로드된 파일을 images 폴더로 이동
for fname in uploaded.keys():
    os.rename(fname, f"donut_dataset/images/{fname}")

print("📁 donut_dataset/images/ 에 이미지 정리됨!")


📁 donut_dataset/images/ 에 이미지 정리됨!


파일명 기반 JSON 라벨 자동 생성

In [3]:
import os
import json

IMAGE_DIR = "donut_dataset/images"

def parse_filename(filename):
    name, _ = os.path.splitext(filename)
    parts = name.split("_")

    if len(parts) < 3:
        return None

    date_raw = parts[0]  # YYYYMMDD
    store = parts[1]     # 상호명
    total = parts[2]     # 금액

    # YYYY-MM-DD 변환
    date = f"{date_raw[:4]}-{date_raw[4:6]}-{date_raw[6:8]}"

    return {
        "date": date,
        "store": store,
        "total": total
    }

labels = {}

for fname in os.listdir(IMAGE_DIR):
    if fname.lower().endswith((".jpg", ".jpeg", ".png")):
        info = parse_filename(fname)
        if info:
            labels[fname] = f'<s>{{"date":"{info["date"]}","store":"{info["store"]}","total":"{info["total"]}"}}</s>'
        else:
            print("⚠ 형식 잘못된 파일:", fname)

# labels.json 저장
with open("donut_dataset/labels.json", "w", encoding="utf-8") as f:
    json.dump(labels, f, ensure_ascii=False, indent=4)

print("✨ labels.json 생성 완료!")


✨ labels.json 생성 완료!


labels.json 로컬 PC에 다운로드

In [4]:
from google.colab import files
files.download("donut_dataset/labels.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

영수증 이미지 증강 (데이터 개수 늘림)

In [5]:
import os
import cv2
import random
import numpy as np

SRC_DIR = "donut_dataset/images"
AUG_DIR = "donut_dataset/images_aug"
os.makedirs(AUG_DIR, exist_ok=True)

def random_rotation(img):
    angle = random.uniform(-5, 5)
    h, w = img.shape[:2]
    M = cv2.getRotationMatrix2D((w//2, h//2), angle, 1)
    return cv2.warpAffine(img, M, (w, h), borderMode=cv2.BORDER_REPLICATE)

def random_perspective(img):
    h, w = img.shape[:2]
    margin = 15
    pts1 = np.float32([[0,0],[w,0],[0,h],[w,h]])
    pts2 = np.float32([
        [random.uniform(-margin, margin), random.uniform(-margin, margin)],
        [w + random.uniform(-margin, margin), random.uniform(-margin, margin)],
        [random.uniform(-margin, margin), h + random.uniform(-margin, margin)],
        [w + random.uniform(-margin, margin), h + random.uniform(-margin, margin)]
    ])
    M = cv2.getPerspectiveTransform(pts1, pts2)
    return cv2.warpPerspective(img, M, (w, h), borderMode=cv2.BORDER_REPLICATE)

def adjust_light(img):
    alpha = random.uniform(0.9, 1.1)
    beta = random.uniform(-15, 15)
    return cv2.convertScaleAbs(img, alpha=alpha, beta=beta)

def augment_and_save(img, filename, idx):
    aug_img = img.copy()

    # 3개의 랜덤 augmentation 적용
    if random.random() < 0.7:
        aug_img = random_rotation(aug_img)
    if random.random() < 0.7:
        aug_img = random_perspective(aug_img)
    if random.random() < 0.7:
        aug_img = adjust_light(aug_img)

    # 증강 파일명
    new_name = f"aug{idx}_{filename}"
    cv2.imwrite(os.path.join(AUG_DIR, new_name), aug_img)

# 원본 이미지 리스트
image_files = [f for f in os.listdir(SRC_DIR) if f.lower().endswith((".jpg", ".png", ".jpeg"))]

print("총 원본 이미지 수:", len(image_files))

# 이미지당 5개 증강 생성
for filename in image_files:
    path = os.path.join(SRC_DIR, filename)
    img = cv2.imread(path)

    if img is None:
        print("⚠ 이미지 불러오기 실패:", filename)
        continue

    for i in range(1, 6):  # 5개씩 생성
        augment_and_save(img, filename, i)

print("✨ 증강 완료! images_aug 폴더 확인해보세요.")


총 원본 이미지 수: 30
✨ 증강 완료! images_aug 폴더 확인해보세요.


In [6]:
from google.colab import files

# donut_dataset 전체 압축 (images, images_aug, labels.json 모두 포함)
!zip -r donut_dataset.zip donut_dataset

# PC로 다운로드
files.download("donut_dataset.zip")


  adding: donut_dataset/ (stored 0%)
  adding: donut_dataset/images_aug/ (stored 0%)
  adding: donut_dataset/images_aug/aug2_20251118_매머드익스프레스_3400.jpg (deflated 7%)
  adding: donut_dataset/images_aug/aug5_20251119_emarteveryday_8680.jpg (deflated 5%)
  adding: donut_dataset/images_aug/aug2_20251127_명랑핫도그_7200.jpg (deflated 1%)
  adding: donut_dataset/images_aug/aug4_20251126_대한왕뼈구이감자탕_40000.jpg (deflated 3%)
  adding: donut_dataset/images_aug/aug1_20251118_몬스터플레이스_22000.jpg (deflated 0%)
  adding: donut_dataset/images_aug/aug2_20251118_맥도날드_4600.jpg (deflated 0%)
  adding: donut_dataset/images_aug/aug4_20251202_알촌_7300.jpg (deflated 1%)
  adding: donut_dataset/images_aug/aug2_20251120_CU_2700.jpg (deflated 5%)
  adding: donut_dataset/images_aug/aug2_20251129_CU_2400.jpg (deflated 6%)
  adding: donut_dataset/images_aug/aug5_20251118_몬스터플레이스_22000.jpg (deflated 0%)
  adding: donut_dataset/images_aug/aug4_20251123_캔모아_29000.jpg (deflated 3%)
  adding: donut_dataset/images_aug/aug5_202511

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

증강 파일 확인

In [7]:
!ls donut_dataset/images_aug

 aug1_20251113_알촌_8300.jpg
 aug1_20251117_죠스아이스크림_1400.jpg
 aug1_20251117_죠스아이스크림_1500.jpg
 aug1_20251118_매머드익스프레스_1100.jpg
 aug1_20251118_몬스터플레이스_22000.jpg
 aug1_20251118_매머드익스프레스_3400.jpg
 aug1_20251118_맥도날드_4200.jpg
 aug1_20251118_맥도날드_4600.jpg
 aug1_20251118_은하수식당_9000.jpg
 aug1_20251119_몬스터플레이스_11000.jpg
 aug1_20251119_emarteveryday_8680.jpg
 aug1_20251120_CU_2400.jpg
 aug1_20251120_CU_2700.jpg
 aug1_20251123_캔모아_29000.jpg
 aug1_20251123_맥도날드_9100.jpg
 aug1_20251125_몬스터플레이스_10500.jpg
 aug1_20251125_명랑핫도그_7200.jpg
 aug1_20251126_대한왕뼈구이감자탕_40000.jpg
 aug1_20251126_매머드익스프레스_4500.jpg
 aug1_20251127_동경규동_13900.jpg
 aug1_20251127_동경규동_32700.jpg
 aug1_20251127_명랑핫도그_7200.jpg
 aug1_20251127_동경규동_8400.jpg
 aug1_20251127_CU_10400.jpg
 aug1_20251127_CU_4100.jpg
 aug1_20251128_CU_4000.jpg
 aug1_20251129_CU_2400.jpg
 aug1_20251202_알촌_6000.jpg
 aug1_20251202_알촌_7300.jpg
'aug1_20251203_BURGER KING_9700.jpg'
 aug2_20251113_알촌_8300.jpg
 aug2_20251117_죠스아이스크림_1400.jpg
 aug2_20251117_죠스아이스크림_1500.jp